In [10]:
## Import whatever module needed
import math, json
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
# from pyproj import proj
import pandas as pd
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum
import requests

In [2]:
## Variables
apiToken = "689a52b9420360bef6af63ad889bcbf4cb34f26e1d1348b2d3a2ff3b71dfe6828d07395724415aa9"
urlBase = 'https://api.fulcrumapp.com/api/v2/'
fulcrum = Fulcrum(key=apiToken)
osTimestamp = datetime.today() - timedelta(days=4)
recordsPerPage = 5000
print(fulcrum)

In [3]:
## Data CTI
appID = "0092aa7f-a2e8-4972-8f26-46eb98b23a12" # ID of Combined Tree Invnetory
formdataSource = fulcrum.forms.find(appID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountSource = fulcrum.forms.find(appID)['form']['record_count']
pagesSource = math.ceil(recordCountSource / recordsPerPage)

## get data I
if pagesSource > 1:
    for p in range(1, pagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': appID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dataSource.extend(dataPage)
        else:
            dataSource = dataPage
else:
    dataSource = fulcrum.records.search(url_params={'form_id': appID, 'page': 1, 'per_page': recordsPerPage})['records']

In [7]:
## What do we have? Attributes:
type(dataSource)
cticolumns = list(dataSource[0].keys())
cticolumns[13:25]  

['created_duration',
 'updated_duration',
 'edited_duration',
 'form_id',
 'project_id',
 'record_series_id',
 'assigned_to',
 'assigned_to_id',
 'form_values',
 'latitude',
 'longitude',
 'altitude']

In [8]:
print(dataSource[0]["form_values"])
status_values = {record["id"] for record in dataSource if "id" in record}
print(len(status_values))
print(dataSource[0]["form_values"]["5cf5"])
print(dataSource[0]["form_values"]["5cf5"][0]["photo_id"])

{'2c1f': 'Campsite,Irrigation,trees watered', '6e5e': {'choice_values': ['BP inventory'], 'other_values': []}, '3fee': {'choice_values': ['Acacia sp. - السنط'], 'other_values': []}, '5cf5': [{'photo_id': '055e3eac-2472-42e5-93ed-e9ea9974b4c2', 'caption': ''}]}
253855
[{'photo_id': '055e3eac-2472-42e5-93ed-e9ea9974b4c2', 'caption': ''}]
055e3eac-2472-42e5-93ed-e9ea9974b4c2


In [11]:
# Test API access with a known photo 
test_photo_id = dataSource[0]['form_values']['5cf5'][0]['photo_id']
print("Test Photo ID:", test_photo_id)
test_url = f"https://api.fulcrumapp.com/api/v2/photos/{test_photo_id}.json"
response = requests.get(test_url, headers={'X-ApiToken': apiToken})
print("Test Response:", response.status_code)
print("Test Data:", response.json())

Test Photo ID: 055e3eac-2472-42e5-93ed-e9ea9974b4c2
Test Response: 200
Test Data: {'photo': {'created_by': '[BP] admin L2', 'created_by_id': 'f8161492-3026-4f3c-a770-93f8e540d4d1', 'updated_by': '[BP] admin L2', 'updated_by_id': 'f8161492-3026-4f3c-a770-93f8e540d4d1', 'thumbnail': 'https://content.fulcrumapp.com/photos/thumb_d3ac2952-eac8-42fe-9df0-04c46d90546b-055e3eac-2472-42e5-93ed-e9ea9974b4c2.jpg?Expires=1750771704&Signature=kL3fDbD-M0b25lUwHx2xzo~7YvSzD~mLiTtpNihIK6OEGfafCdIOT~0jfSj20TgpumajX4-n5P-4K9eUoiifIY7F2rydpSW9rA46UKK6~WUhh2B2lZ6ls0wL2XEL0PFe8DWNbKCU1s34TZ7uHlLieqpGEm8Uyu-557GePzvYpnNOlmLVD-M1xDZyBMbsQC0v03S-MYgsWVMZCqIdfogAY-xXMrpzOcWgV~w6n16ER5PMc5tLeL6X~mk07cluIA9w7Qdiebwz~rlGzphqn8jycIdfQWIwDmDTxVbgJhGMGkayjcVoQ6tMpHGxQgl5mllRFbSXrLUK8BnYVdV8v-qYcg__&Key-Pair-Id=K1J5KALR9YAJQS', 'large': 'https://content.fulcrumapp.com/photos/large_d3ac2952-eac8-42fe-9df0-04c46d90546b-055e3eac-2472-42e5-93ed-e9ea9974b4c2.jpg?Expires=1750771704&Signature=KyzzeYzsdjLD0QE-zqTBfWcGAOcd573

In [ ]:
import os
import time
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor

def download_fulcrum_photos(dataSource, output_dir = r"D:\BPLA Dropbox\03 Planning\1232-T2-TM2_1-GIS-Remote-Sensing\06_GIS-Data\03_TreeInventory\photos"):
    """
    Downloads photos using the content URLs from Fulcrum API response
    Args:
        dataSource: The complete dataset from fulcrum.records.search()
        output_dir: Directory to save photos
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"💾 Saving photos to: {os.path.abspath(output_dir)}")
    
    # Extract all photo references from the 5cf5 field
    photo_references = []
    for record in dataSource:
        if 'form_values' in record and '5cf5' in record['form_values']:
            photos = record['form_values']['5cf5']
            if isinstance(photos, list):
                for photo in photos:
                    if 'photo_id' in photo:
                        photo_references.append({
                            'photo_id': photo['photo_id'],
                            'record_id': record['id'],
                            'caption': photo.get('caption', '')
                        })
    
    print(f"📸 Found {len(photo_references)} photo references in '5cf5' field")
    
    # Prepare download tasks (skip existing files)
    download_tasks = []
    for ref in photo_references:
        filename = f"{ref['photo_id']}.jpg"
        filepath = os.path.join(output_dir, filename)
        
        if not os.path.exists(filepath):
            download_tasks.append({
                'photo_id': ref['photo_id'],
                'path': filepath,
                'record_id': ref['record_id']
            })

    if not download_tasks:
        print("✅ All photos already downloaded")
        return

    print(f"📥 Preparing to download {len(download_tasks)} new photos")

    # Download function using content URLs
    def download_single(task):
        try:
            # First get photo metadata to obtain content URL
            metadata_url = f"https://api.fulcrumapp.com/api/v2/photos/{task['photo_id']}.json"
            headers = {'X-ApiToken': apiToken}
            
            for attempt in range(3):  # Retry up to 3 times
                try:
                    # Get photo metadata
                    response = requests.get(metadata_url, headers=headers)
                    if response.status_code != 200:
                        time.sleep(2 ** attempt)
                        continue
                        
                    photo_data = response.json().get('photo', {})
                    if not photo_data:
                        time.sleep(2 ** attempt)
                        continue
                    
                    # Use the original content URL
                    download_url = photo_data.get('original')
                    if not download_url:
                        time.sleep(2 ** attempt)
                        continue
                    
                    # Download the photo (no auth needed for content URLs)
                    photo_response = requests.get(download_url, stream=True)
                    
                    if photo_response.status_code == 200:
                        # Write directly to disk
                        with open(task['path'], 'wb') as f:
                            for chunk in photo_response.iter_content(1024):
                                f.write(chunk)
                        
                        # Verify it's a valid JPG
                        with open(task['path'], 'rb') as f:
                            if f.read(2) == b'\xff\xd8':
                                return True
                            else:
                                os.remove(task['path'])
                                raise ValueError("Invalid JPG file")
                    else:
                        time.sleep(2 ** attempt)
                        
                except Exception as e:
                    time.sleep(2 ** attempt)
                    if attempt == 2:
                        print(f"❌ Failed to download {task['photo_id']}: {str(e)}")
                        return False
                        
            return False
            
        except Exception as e:
            print(f"❌ Error downloading {task['photo_id']}: {str(e)}")
            return False

    # Download with progress (using 4 workers)
    success = 0
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = list(tqdm(
            executor.map(download_single, download_tasks),
            total=len(download_tasks),
            desc="Downloading photos"
        ))
        success = sum(results)

    print(f"\n✅ Completed! Downloaded {success}/{len(download_tasks)} photos")
    print(f"💾 Saved to: {os.path.abspath(output_dir)}")
    
    # Generate failure report
    if success < len(download_tasks):
        failed = len(download_tasks) - success
        print(f"\n⚠️ Failed to download {failed} photos")
        
        # Save failed IDs to file
        failed_file = os.path.join(output_dir, 'failed_downloads.txt')
        with open(failed_file, 'w') as f:
            for task, result in zip(download_tasks, results):
                if not result:
                    f.write(f"{task['photo_id']}\n")
        
        print(f"Saved list of failed IDs to: {failed_file}")

# Run the downloader
download_fulcrum_photos(dataSource)

c:\Users\User3\miniconda3\envs\py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


💾 Saving photos to: D:\BPLA Dropbox\03 Planning\1232-T2-TM2_1-GIS-Remote-Sensing\06_GIS-Data\03_TreeInventory\photos
📸 Found 98435 photo references in '5cf5' field
📥 Preparing to download 98435 new photos
